# Исследование надежности заемщиков


**Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.**

**Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.**

## Откройте таблицу и изучите общую информацию о данных

**Задание 1. Импортируйте библиотеку pandas. Считайте данные из csv-файла в датафрейм и сохраните в переменную `data`. Путь к файлу:**

`/datasets/data.csv`

In [91]:
import pandas as pd
import os
from pathlib import Path

In [92]:
ROOT_DIR = Path(os.path.abspath("__file__")).parent.parent

In [93]:
try:
    data = pd.read_csv(ROOT_DIR / 'datasets/data.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')

**Задание 2. Выведите первые 20 строчек датафрейма `data` на экран.**

In [94]:
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


**Задание 3. Выведите основную информацию о датафрейме с помощью метода `info()`.**

In [95]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


***На первый взгляд колонки заполнены корректро, следует лишь изменить тип данных в total_income - "Доходы". Переходим к поиску пропусков.***

## Предобработка данных

### Удаление пропусков

**Задание 4. Выведите количество пропущенных значений для каждого столбца. Используйте комбинацию двух методов.**

In [96]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

**Задание 5. В двух столбцах есть пропущенные значения. Один из них — `days_employed`. Пропуски в этом столбце вы обработаете на следующем этапе. Другой столбец с пропущенными значениями — `total_income` — хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполнить пропуски в этом столбце нужно медианным значением по каждому типу из столбца `income_type`. Например, у человека с типом занятости `сотрудник` пропуск в столбце `total_income` должен быть заполнен медианным доходом среди всех записей с тем же типом.**

In [97]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### Обработка аномальных значений

**Задание 6. В данных могут встречаться артефакты (аномалии) — значения, которые не отражают действительность и появились по какой-то ошибке. таким артефактом будет отрицательное количество дней трудового стажа в столбце `days_employed`. Для реальных данных это нормально. Обработайте значения в этом столбце: замените все отрицательные значения положительными с помощью метода `abs()`.**

In [98]:
data['days_employed'] = data['days_employed'].abs()

**Задание 7. Для каждого типа занятости выведите медианное значение трудового стажа `days_employed` в днях.**

In [99]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения сложно, поэтому оставьте их как есть. Тем более этот столбец не понадобится вам для исследования.

**Задание 8. Выведите перечень уникальных значений столбца `children`.**

In [100]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

**Задание 9. В столбце `children` есть два аномальных значения. Удалите строки, в которых встречаются такие аномальные значения из датафрейма `data`.**

In [101]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

**Задание 10. Ещё раз выведите перечень уникальных значений столбца `children`, чтобы убедиться, что артефакты удалены.**

In [102]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5])

### Удаление пропусков (продолжение)

**Задание 11. Заполните пропуски в столбце `days_employed` медианными значениями по каждого типа занятости `income_type`.**

In [103]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

**Задание 12. Убедитесь, что все пропуски заполнены. Проверьте себя и ещё раз выведите количество пропущенных значений для каждого столбца с помощью двух методов.**

In [104]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Изменение типов данных

**Задание 13. Замените вещественный тип данных в столбце `total_income` на целочисленный с помощью метода `astype()`.**

In [105]:
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов

**Задание 14. Обработайте неявные дубликаты в столбце `education`. В этом столбце есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Приведите их к нижнему регистру. Проверьте остальные столбцы.**

In [106]:
data['education'] = data['education'].str.lower()

**Задание 15. Выведите на экран количество строк-дубликатов в данных. Если такие строки присутствуют, удалите их.**

In [107]:
data.duplicated().sum()

71

In [108]:
data = data.drop_duplicates()

### Категоризация данных

**Задание 16. На основании диапазонов, указанных ниже, создайте в датафрейме `data` столбец `total_income_category` с категориями:**

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.


**Например, кредитополучателю с доходом 25000 нужно назначить категорию `'E'`, а клиенту, получающему 235000, — `'B'`. Используйте собственную функцию с именем `categorize_income()` и метод `apply()`.**

In [109]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [110]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

**Задание 17. Выведите на экран перечень уникальных целей взятия кредита из столбца `purpose`.**

In [111]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

**Задание 18. Создайте функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:**

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

**Например, если в столбце `purpose` находится подстрока `'на покупку автомобиля'`, то в столбце `purpose_category` должна появиться строка `'операции с автомобилем'`.**

**Используйте собственную функцию с именем `categorize_purpose()` и метод `apply()`. Изучите данные в столбце `purpose` и определите, какие подстроки помогут вам правильно определить категорию.**

In [112]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [113]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

### Шаг 3. Исследуйте данные и ответьте на вопросы

#### 3.1 Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [114]:
data.pivot_table(index='children', values='debt', aggfunc=['mean', 'count'])

,mean,count
,debt,debt
children,,
0,0.075438,14091
1,0.092346,4808
2,0.094542,2052
3,0.081818,330
4,0.097561,41
5,0.000000,9


**Вывод:** По статистике данного банка заёмщики, имеющие пятерых детей вносят платежи в срок, но данные учитывать не будем, так как для вывода не хватает количества таких заёмщиков - их 9. Заёмщики, имеющие от одного до четырёх детей находятся примерно на одном уровне - практически 10% из них совершали просрочки. А вот бездетные заёмщики банка выдали самый низкий показатель среди неплательщиков - 7,5%.

#### 3.2 Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [115]:
data.pivot_table(index='family_status', values='debt', aggfunc=['mean', 'count'])

,mean,count
,debt,debt
family_status,,
Не женат / не замужем,0.097639,2796
в разводе,0.070648,1189
вдовец / вдова,0.066246,951
гражданский брак,0.093130,4134
женат / замужем,0.075606,12261


**Вывод:** Из сводной таблицы видим, что более 9% неуплат приходится на заёмщиков, которые не состояли в браке либо проживают совместно неофициально. Заёмщики же состоящие в браке, вдовцы и разведённые показали схожий результат - 6,6 - 7,6% просрочек по платежам. ***Заёмщики состоящие или состоявшие в браке более надежны.***

#### 3.3 Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [116]:
data.pivot_table(index='total_income_category', values='debt', aggfunc=['mean', 'count'])

,mean,count
,debt,debt
total_income_category,,
A,0.080000,25
B,0.070602,5014
C,0.084982,15921
D,0.060172,349
E,0.090909,22


**Вывод:** Если сравнить только две самые большие категории C (50 - 200) и B (200 - 1 млн), выходит что заёмщики с более высоким доходом являются более надежными.

Напрашивается рекомендация банку - изменить градацию по доходам для получения более сбалансированных выборок.

#### 3.4 Как разные цели кредита влияют на его возврат в срок?

In [117]:
data.pivot_table(index='purpose_category', values='debt', aggfunc=['mean', 'count'])

,mean,count
,debt,debt
purpose_category,,
операции с автомобилем,0.093480,4279
операции с недвижимостью,0.072551,10751
получение образования,0.092528,3988
проведение свадьбы,0.079118,2313


**Вывод:** наиболее проблемными выглядят автокредиты, а надежными - кредиты на недвижимость.

#### 3.5 Приведите возможные причины появления пропусков в исходных данных.

Либо человеческий фактор (кто-то что-то не ввел, умышленно или нет), либо технический, например некорректная выгрузка из БД

#### 3.6 Объясните, почему заполнить пропуски медианным значением — лучшее решение для количественных переменных.

*Ответ:* С помощью медианы мы видим центральное значение из всех данных, в отличии от среднего, которое может быть необъективным из-за выбросов.
***Медиана менее чувствительна к выбросам***

### Шаг 4: общий вывод.

<div style="border:solid blue 3px; padding: 20px">
    <div class="alert-info">
        <p>Мы исследовали зависимости определенных факторов на своевременную уплату долга.</p>
    <p><b>Зависимость от количества детей:</b> бездетные заёмщики банка показывают самый низкий показатель среди неплательщиков - 7,5%. Заёмщики, имеющие 1-4 детей находятся примерно на одном уровне - практически 10% из них совершали просрочки.</p>
    <p><b>Зависимость от семейного положения:</b> из сводной таблицы видим, что более 9% неуплат приходится на заёмщиков, которые не состояли в браке либо проживают совместно неофициально. Заёмщики же состоящие в браке, вдовцы и разведённые показали схожий результат - 6,6 - 7,6% просрочек по платежам.</p>
    <p><b>Зависимость от уровня дохода:</b> Для граждан с минимальным доходом - до 30'000 рублей и максимальным - более 1 млн рублей мы не можем сделать вывод из-за слишком небольшого количества кредитов у потребителей с такими доходами в данном банке. Самый низкий процент просрочек у заёмщиков с доходом от 30'000 до 50'000 рублей - у них 6%. За ними следуют заёмщики с доходом от 200'000 до 1 млн рублей - не вовремя оплатили около 7%. И с показателем просрочек в 8,5% в данном банке заёмщики с доходом 50'000-200'000 рублей, но таких потребителей в разы больше. Это указывает нам на то, что выборка зависимости между уровнем дохода и возвратом кредита в срок в данном случае некорректна.</p>
    <p><b>Зависимость от цели кредита:</b> выяснили, что неуплата долга напрямую не зависит от цели кредитования - показатели находятся примерно на одном уровне: чаще не выплачивали долг вовремя заёмщики, взявшие средства на образование и операции с автомобилем - 9,2 - 9,3%; от 7,2% до 7,9% заёмщиков не вернули в срок средства, взятые на операции с недвижимостью и на проведение свадьбы.</p>
    <p> __По любому из взятых нами факторов мы видим показатели ниже 10% в отношении просроченных платежей, что говорит нам о корректности исследования и отсутствии каких-либо выбросов в показателях.__ </p>
    </div>
</div>